# 🎭 Week 11: LLM Orchestration with Agents and Chains

This notebook covers building complex AI systems with chains, agents, and tools.

## Table of Contents
1. [Orchestration Fundamentals](#1-orchestration-fundamentals)
2. [Chains](#2-chains)
3. [Tools](#3-tools)
4. [Agents](#4-agents)
5. [Memory](#5-memory)
6. [Building a RAG Agent](#6-building-a-rag-agent)

---

In [ ]:
# Setup
import sys
sys.path.insert(0, '../..')

from src.orchestration import (
    Chain,
    SequentialChain,
    ParallelChain,
    Agent,
    ReActAgent,
    Tool,
    ToolRegistry,
    Memory,
    ConversationMemory,
)
from src.orchestration.orchestration import (
    LambdaChain, ConditionalChain, Message, MessageRole,
    CalculatorTool, SearchTool
)

print("✅ Setup complete!")

---

## 1. Orchestration Fundamentals

### 1.1 Why Orchestration?

Complex AI applications require:
- **Multi-step reasoning** - Break complex tasks into steps
- **Tool usage** - Access external systems (search, calculate, etc.)
- **Memory** - Maintain context across interactions
- **Error handling** - Graceful failure recovery

### 1.2 Core Concepts

```
┌─────────────────────────────────────────────────────────┐
│                    ORCHESTRATION                        │
├─────────────┬─────────────┬─────────────┬──────────────┤
│   CHAINS    │   AGENTS    │   TOOLS     │   MEMORY     │
│  (compose)  │  (reason)   │  (act)      │  (remember)  │
└─────────────┴─────────────┴─────────────┴──────────────┘
```

---

## 2. Chains

### 2.1 Sequential Chains

Run steps in sequence, output of one becomes input to next.

In [ ]:
# Create processing pipeline
pipeline = SequentialChain([
    LambdaChain(lambda x: x.strip().lower(), name="normalize"),
    LambdaChain(lambda x: x.replace("  ", " "), name="clean_spaces"),
    LambdaChain(lambda x: x.title(), name="title_case"),
])

# Test
input_text = "  HELLO   WORLD  "
result = pipeline.run(input_text)

print(f"Input:  '{input_text}'")
print(f"Output: '{result}'")
print("\n✅ Text normalized through 3-step pipeline!")

In [ ]:
# Example: Document Processing Pipeline
def extract_keywords(text):
    """Extract key terms from text."""
    words = text.lower().split()
    stopwords = {'the', 'is', 'a', 'an', 'and', 'or', 'to', 'of', 'in'}
    keywords = [w for w in words if w not in stopwords and len(w) > 3]
    return list(set(keywords))[:5]

def summarize(keywords):
    """Create simple summary from keywords."""
    return f"Document about: {', '.join(keywords)}"

doc_pipeline = SequentialChain([
    LambdaChain(extract_keywords, name="extract"),
    LambdaChain(summarize, name="summarize"),
])

doc = "Machine learning is a subset of artificial intelligence that enables computers to learn from data."
summary = doc_pipeline.run(doc)

print(f"Document: {doc[:50]}...")
print(f"Summary:  {summary}")

### 2.2 Parallel Chains

Run multiple chains simultaneously on the same input.

In [ ]:
# Parallel analysis
parallel = ParallelChain([
    LambdaChain(lambda x: len(x), name="count_chars"),
    LambdaChain(lambda x: len(x.split()), name="count_words"),
    LambdaChain(lambda x: x.upper(), name="uppercase"),
])

text = "Hello world from parallel chains"
results = parallel.run(text)

print(f"Input: '{text}'")
print(f"\nResults:")
print(f"  Char count:  {results[0]}")
print(f"  Word count:  {results[1]}")
print(f"  Uppercase:   {results[2]}")

### 2.3 Conditional Chains

Route to different chains based on conditions.

In [ ]:
# Conditional routing
router = ConditionalChain(
    condition=lambda x: len(x) > 20,
    if_true=LambdaChain(lambda x: f"LONG: {x[:20]}..."),
    if_false=LambdaChain(lambda x: f"SHORT: {x}")
)

short_text = "Hello!"
long_text = "This is a much longer piece of text that exceeds the limit."

print(f"Short: {router.run(short_text)}")
print(f"Long:  {router.run(long_text)}")

---

## 3. Tools

### 3.1 Creating Custom Tools

In [ ]:
from src.orchestration.orchestration import Tool, ToolResult

class WeatherTool(Tool):
    """Tool to get weather information."""
    
    def __init__(self):
        super().__init__(
            name="weather",
            description="Get current weather for a location",
            parameters={"location": "City name or coordinates"}
        )
    
    def run(self, location: str = "", **kwargs) -> ToolResult:
        # Mock weather data
        weather_data = {
            "london": "Cloudy, 12°C",
            "new york": "Sunny, 22°C",
            "tokyo": "Rainy, 18°C",
        }
        
        location_lower = location.lower()
        if location_lower in weather_data:
            return ToolResult(output=weather_data[location_lower], success=True)
        else:
            return ToolResult(
                output=None, 
                success=False, 
                error=f"Weather data not available for {location}"
            )

class DatabaseTool(Tool):
    """Tool to query a database."""
    
    def __init__(self, data: dict):
        super().__init__(
            name="database",
            description="Query information from the database",
            parameters={"query": "Search query"}
        )
        self.data = data
    
    def run(self, query: str = "", **kwargs) -> ToolResult:
        # Simple keyword search
        results = [
            v for k, v in self.data.items() 
            if query.lower() in k.lower()
        ]
        
        if results:
            return ToolResult(output=results, success=True)
        else:
            return ToolResult(output="No results found", success=True)

# Test tools
weather = WeatherTool()
result = weather.run(location="London")
print(f"Weather in London: {result.output}")

### 3.2 Tool Registry

In [ ]:
# Create tool registry
registry = ToolRegistry()

# Register tools
registry.register(CalculatorTool())
registry.register(SearchTool())
registry.register(WeatherTool())

# List available tools
print("Available Tools:")
print(registry.to_prompt())

# Use a tool
calc = registry.get("calculator")
result = calc.run(expression="2 * 3 + 5")
print(f"\nCalculation: 2 * 3 + 5 = {result.output}")

---

## 4. Agents

### 4.1 The ReAct Pattern

ReAct (Reasoning + Acting) agents interleave:
1. **Thought** - Reason about what to do
2. **Action** - Use a tool
3. **Observation** - See the result
4. **Repeat** until done

```
Question: What is 25% of 80?

Thought: I need to calculate 25% of 80. I'll use the calculator.
Action: calculator
Action Input: 80 * 0.25
Observation: 20

Thought: I now have the answer.
Action: finish
Action Input: 25% of 80 is 20
```

In [ ]:
# Create ReAct agent
agent = ReActAgent(tools=registry)

print("ReAct Agent Initialized")
print("=" * 40)
print(f"Tools: {[t.name for t in registry.list_tools()]}")
print("\nAgent prompt template:")
print(agent.REACT_PROMPT[:300] + "...")

---

## 5. Memory

### 5.1 Conversation Memory

In [ ]:
# Create conversation memory
memory = ConversationMemory(max_messages=10)

# Add messages
memory.add_message(Message(role=MessageRole.USER, content="What is machine learning?"))
memory.add_message(Message(role=MessageRole.ASSISTANT, content="Machine learning is a subset of AI that enables computers to learn from data."))
memory.add_message(Message(role=MessageRole.USER, content="Can you give an example?"))
memory.add_message(Message(role=MessageRole.ASSISTANT, content="A common example is spam email detection."))

# Get context
print("Conversation Context:")
print("=" * 40)
print(memory.to_context())

print(f"\nTotal messages: {len(memory.messages)}")

In [ ]:
# Use memory with chains
def respond_with_context(query, memory):
    """Generate response using conversation context."""
    context = memory.to_context()
    
    # In real implementation, this would use an LLM
    response = f"Based on our conversation about ML and spam detection, here's more info about: {query}"
    
    # Update memory
    memory.add_message(Message(role=MessageRole.USER, content=query))
    memory.add_message(Message(role=MessageRole.ASSISTANT, content=response))
    
    return response

# Test
response = respond_with_context("How accurate is spam detection?", memory)
print(f"Response: {response}")
print(f"\nMemory now has {len(memory.messages)} messages")

---

## 6. Building a RAG Agent

### 6.1 Putting It All Together

In [ ]:
class RAGAgent:
    """
    Complete RAG agent with retrieval, reasoning, and tools.
    """
    
    def __init__(self, documents: list):
        # Initialize components
        self.memory = ConversationMemory(max_messages=20)
        
        # Tools
        self.tools = ToolRegistry()
        self.tools.register(CalculatorTool())
        
        # Simple retriever (use BM25 in production)
        self.documents = documents
    
    def retrieve(self, query: str, top_k: int = 3) -> list:
        """Simple retrieval (production: use BM25/Dense)."""
        query_words = set(query.lower().split())
        scored = []
        
        for doc in self.documents:
            doc_words = set(doc.lower().split())
            score = len(query_words & doc_words)
            scored.append((doc, score))
        
        scored.sort(key=lambda x: -x[1])
        return [doc for doc, _ in scored[:top_k]]
    
    def answer(self, question: str) -> str:
        """Answer question using RAG."""
        # 1. Retrieve relevant context
        context = self.retrieve(question)
        
        # 2. Build prompt
        context_str = "\n".join(f"- {doc}" for doc in context)
        
        # 3. Generate answer (mock - use LLM in production)
        answer = f"Based on the context, {context[0][:50]}..."
        
        # 4. Update memory
        self.memory.add_message(Message(role=MessageRole.USER, content=question))
        self.memory.add_message(Message(role=MessageRole.ASSISTANT, content=answer))
        
        return {
            "answer": answer,
            "context": context,
            "sources": len(context)
        }

# Test RAG agent
documents = [
    "Machine learning enables computers to learn from data.",
    "Deep learning uses neural networks with many layers.",
    "Natural language processing helps computers understand text.",
]

rag = RAGAgent(documents)
result = rag.answer("What is machine learning?")

print("RAG Agent Response:")
print("=" * 40)
print(f"Answer: {result['answer']}")
print(f"Sources used: {result['sources']}")

---

## 📝 Summary

### Key Concepts

| Component | Purpose | Example |
|-----------|---------|--------|
| **Chain** | Compose processing steps | Sequential, Parallel |
| **Tool** | Extend agent capabilities | Calculator, Search |
| **Agent** | Autonomous reasoning | ReAct pattern |
| **Memory** | Maintain context | Conversation history |

### Best Practices

1. **Keep chains simple** - Single responsibility per step
2. **Make tools atomic** - One clear purpose per tool
3. **Limit agent iterations** - Prevent infinite loops
4. **Manage memory size** - Use sliding window
5. **Handle errors gracefully** - Tools should not crash agents